In [1]:
import glob
import pandas as pd
import os
import glob

# Personnal Import 
from utilities import get_mode_date2path
from load_data import load_subway_15_min,load_CRITER,load_netmob

# Data
- Validation individuelles, aggrégée **3min**
- Metro (entrée/sortie)  **15 min**
- Sensor, aggrégé **6 min**

### Notes et questionnements
- Un 'VAL_ARRET_CODE' peut être l'arrêt de plusieurs mêmes bus, voir d'un même bus et d'un même arrêt de métro. Où d'un même bus et d'un même arrêt de tram. Je dois donc nommer différement les VAL_ARRET_CODE de chacun des modes. Une proposition est de mettre le mode (B,S,T) devant les id. Comme ça on pourra regrouper sans soucis.
- La moyenne des déplacement de la df_subway est de 5 trajet toute les 3 minutes, quelque soit la station et l'heure (d'ouverture) considéré. Max 88.
- Ok on a aggrégé 3 min, mais est-ce qu'on peut recouper les sorties 3min avec les validation + Sortie de métro 15 min? 

In [2]:
# Subway 15 Min :                                   [J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,~, , ,~,~, , ],[J,F,M, , , , , , , , , ]
# Validation Individuelles  Nov 2019 - Mai 2021     [ , , , , , , , , , ,N,D],[J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M, , , , , , , ]
# NetMob 15 Min :                                   [ , ,M,A,M, , , , , , , ],[ , , , , , , , , , , , ],[ , , , , , , , , , , , ]
# Sensor 6 Min :  Janvier 2019 - Decembre 2020      [J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,J,J,A,S,O,N,D]

## Load Validation Individuelles
Load 3 df : df_sub, df_tram, df_bus

In [3]:
folder_path = 'data/'
valid_ind_path = folder_path + 'Sub_Tram_11_2019_03_2020'
dates = ['11-2019','12-2019','1-2020','2-2020','3-2020']

subway_paths, tramway_paths, bus_paths = sorted(glob.glob(os.path.join(valid_ind_path, "*df_subway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_tramway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_bus*.csv")))
mode_month2path = get_mode_date2path([subway_paths,tramway_paths,bus_paths],['sub','tram','bus'])

for name in ['sub','tram','bus']:
    globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])

/var/folders/vc/gn83ctb97rd5lmlbh8djj9fr0000gr/T/ipykernel_9473/2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/var/folders/vc/gn83ctb97rd5lmlbh8djj9fr0000gr/T/ipykernel_9473/2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/var/folders/vc/gn83ctb97rd5lmlbh8djj9fr0000gr/T/ipykernel_9473/2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/var/folders/vc/gn83ctb97rd5lmlbh8djj9fr0000gr/T/ipykernel_9473/2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify 

## Load Subway 15 min

In [10]:
txt_path = "Métro 15 minutes 2019 2020.txt"

df_metro_funi_2019_2020 = load_subway_15_min(folder_path+txt_path)
df_metro_funi_2019_2020.head()

,datetime,Station,Code ligne,in,out
0,2019-01-01 00:00:00,Ampère Victor Hugo,A,2,4.0
1,2019-01-01 00:15:00,Ampère Victor Hugo,A,3,2.0
2,2019-01-01 00:30:00,Ampère Victor Hugo,A,3,7.0
3,2019-01-01 00:45:00,Ampère Victor Hugo,A,1,9.0
4,2019-01-01 01:00:00,Ampère Victor Hugo,A,0,0.0


## Load NetMob 15 min

In [4]:
folder_path = '../../Data/NetMob/'
apps = [app for app in os.listdir(folder_path) if ((app != 'Lyon.geojson') and (not app.startswith('.'))) ]   # Avoid hidden folder and Lyon.geojson

In [5]:
app = apps[0]
folder_days = [day for day in os.listdir(f'{folder_path}/{app}') if (not day.startswith('.')) ] 
day = folder_days[0]
txt_paths = sorted(glob.glob(os.path.join(f'{folder_path}/{app}/{day}', "*.txt")))
txt_paths

['../../Data/NetMob//Apple_Video/20190504/Lyon_Apple_Video_20190504_DL.txt',
 '../../Data/NetMob//Apple_Video/20190504/Lyon_Apple_Video_20190504_UL.txt']

In [6]:
txt_path = txt_paths[0]
df_Apple_Vid_DL = load_netmob(txt_path,day)
df_Apple_Vid_DL.head()

,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
tile_id,,,,,,,,,,,,,,,,,,,,,
66,1834,1596,1378,1575,637,421,203,1366,547,254,...,4114,2992,2887,3297,4634,4208,4886,2095,2066,919
353,2000,1687,1357,1548,764,484,195,2492,591,407,...,4673,3288,3156,3778,5123,4400,5300,2187,2176,984
354,1621,1540,1258,1381,490,375,94,1131,453,302,...,3887,3128,3048,3337,4582,3792,4519,2189,2309,984
640,1803,1493,1201,1332,632,514,73,2951,428,370,...,4404,3352,3082,3512,4697,4056,4862,2155,2156,1074
641,1621,1583,1296,1383,493,415,75,1087,402,368,...,4064,3426,3206,3517,4834,4011,4719,2256,2230,1073


## Load Sensor 6 min

In [2]:
years = [2019,2020]
folder_path = '../../Data/Comptages_Velo_Routier/CRITER/'
list_files = sorted(glob.glob(os.path.join(f'{folder_path}6 min {years[0]}', "*.txt"))) + sorted(glob.glob(os.path.join(f'{folder_path}6 min {years[1]}', "*.txt")))
oct_2019_mar_2020 = list_files[9:15]

In [3]:
df = load_CRITER(oct_2019_mar_2020[0])
len_init = len(df)
print((list_files[0]))
df.head()

/Users/romainrochas/Desktop/Codes/Cleaned_Code/prediction validation/load_data.py:36: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(txt_path, sep=';',infer_datetime_format=True)


../../Data/Comptages_Velo_Routier/CRITER/6 min 2019/6mn_01_Janvier_2019.txt


,ID_POINT_MESURE,NOM_POINT_MESURE,DEBIT_HEURE,TAUX_HEURE,HORODATE,NOMBRE_ECH_1_MIN_MANQUANTS,day,str_hour_min,hour_min
0,1,Berthelot_Servant_Mace,380,3,2019-10-01 00:00:00,0,1,0:0,0.0
1,1,Berthelot_Servant_Mace,240,2,2019-10-01 00:06:00,0,1,0:6,1.0
2,1,Berthelot_Servant_Mace,270,2,2019-10-01 00:12:00,0,1,0:12,2.0
3,1,Berthelot_Servant_Mace,270,2,2019-10-01 00:18:00,0,1,0:18,3.0
4,1,Berthelot_Servant_Mace,200,1,2019-10-01 00:24:00,0,1,0:24,4.0


# 1er Etape : Prédiction Métro
- On va d'abord prédire la demande sur une ligne (disons A).  
- On va comparer des modèle : LSTM, CNN, CNN-LSTM, GNN.
    - A priori pas de "raison" que le GNN marche mieux. Si c'est le cas, c'est peut être simplement que le modèle est plus complexe, mais j'ai du mal à croire que si on donne les bonnes informations (historique -7d, -1d, -4,3,2,1t), on a des meilleurs résultats avec GNN. Sauf si il y a des relation asynchrone "récurrentes", mais sans causalité. De la même manière que l'historique -7d sert de référence, mais ne témoigne pas d'un lien causal.
- Identifier des moments interessants : anomalies sur entrée/sortie métro. Voir les prédictions sur ces moments là particulier.
